In [4]:
from code_tokenizer import *

code = """
posicao.Trigger == "DEACTIVATION" || posicao.Trigger == "POWER_LOSS"
"""

code = """
my_list = [1, 2]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)
"""

print(color_by_char_type(code))


for i in math_layer(code):
    print(i)


my_list = [1, 2]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)

[['assignment', [['word', 'my_list'], [['array_define', [['number', '1'], ['number', '2']]]]]]]
[['assignment', [['word', 'min'], [['word', 'my_list', ['array_call', ['number', '0']]]]]]]
['word', 'for', ['attached_group', [[['assignment', [['word', 'i'], [['number', '1']]]]], [['word', 'i'], ['acess_group', [['word', 'my_list'], ['word', 'length']]], ['symbol', '<']], [['increment', ['word', 'i']]]]], ['attached_block', ['word', 'if', ['attached_group', [['word', 'my_list', ['array_call', ['word', 'i']]], ['word', 'min'], ['symbol', '<']]], ['attached_block', [['assignment', [['word', 'min'], [['word', 'my_list', ['array_call', ['word', 'i']]]]]]]]]]]
[['acess_group', [['word', 'console'], ['word', 'log', ['attached_group', ['word', 'min']]]]]]
